In [2]:
import os
from timeit import default_timer as timer
import pandas as pd
from datetime import datetime, timedelta
from glob import glob
from pathlib import Path
import pandas as pd
import numpy as np
import geopandas as gpd

In [3]:
path_to_data='/scratch/spf248/covid/data'
start_date='01/01/2020'
end_date='04/16/2020'
source='cuebiq'
country='ID'
admin_id='ADM4_PCODE'
city_id='UC_NM_MN'
demographics=[
'wealth_index',
'population_density',
]
day=list(range(8,20))
night=list(range(8))+list(range(20,24))
q_days=0.5
min_pings=20

# Compute share of activity in unique neighborhoods

In [3]:
def get_coords(df):
    df['latitude']=df['point'].apply(lambda x:x['latitude'])
    df['longitude']=df['point'].apply(lambda x:x['longitude'])
    return df.drop('point',1)

In [4]:
print('Import')
start = timer()
data_dirs=glob(os.path.join(path_to_data,source,'processed',country,'pings_geocoded','*'))
pings_geocoded = pd.concat(
    pd.read_parquet(parquet_file)
    for data_dir in data_dirs for parquet_file in Path(data_dir).glob('*.parquet')
)
print("Done in", round(timer()-start), "sec")  

Import
Done in 134 sec


In [5]:
print('Process')
start = timer()
pings_geocoded['date']=pd.to_datetime(pings_geocoded.time.dt.date)
pings_geocoded['hour']=pings_geocoded.time.dt.hour
print("Done in", round(timer()-start), "sec")  

Process
Done in 147 sec


In [6]:
print('Count neighborhood visits')
start = timer()
n_admin_date_id=pings_geocoded.groupby(['date','cuebiq_id'])[admin_id].agg(['nunique','count'])
n_admin_daytime_id=pings_geocoded[pings_geocoded.hour.isin(day)].groupby(['date','cuebiq_id'])[admin_id].agg(['nunique','count'])
n_admin_nighttime_id=pings_geocoded[pings_geocoded.hour.isin(night)].groupby(['date','cuebiq_id'])[admin_id].agg(['nunique','count'])
print("Done in", round(timer()-start), "sec")

Count neighborhood visits
Done in 1184 sec


In [80]:
print('Process')
start = timer()
n_admin_date_id['pct_unique']=n_admin_date_id['nunique'].divide(n_admin_date_id['count'])
n_admin_daytime_id['pct_unique']=n_admin_daytime_id['nunique'].divide(n_admin_daytime_id['count'])
n_admin_nighttime_id['pct_unique']=n_admin_nighttime_id['nunique'].divide(n_admin_nighttime_id['count'])
n_admin_date_id=n_admin_date_id[n_admin_date_id.index.get_level_values('date').isin(pd.date_range(start_date,end_date))].copy()
n_admin_daytime_id=n_admin_daytime_id[n_admin_daytime_id.index.get_level_values('date').isin(pd.date_range(start_date,end_date))].copy()
n_admin_nighttime_id=n_admin_nighttime_id[n_admin_nighttime_id.index.get_level_values('date').isin(pd.date_range(start_date,end_date))].copy()
n_admin_date_id.rename(columns={'count':'n_pings'},inplace=True)
n_admin_daytime_id.rename(columns={'count':'n_pings'},inplace=True)
n_admin_nighttime_id.rename(columns={'count':'n_pings'},inplace=True)
print("Done in", round(timer()-start), "sec")

In [8]:
print('Save')
start = timer()
n_admin_date_id.to_csv(os.path.join(path_to_data,source,'processed',country,'n_admin_date_id.csv'))
n_admin_daytime_id.to_csv(os.path.join(path_to_data,source,'processed',country,'n_admin_daytime_id.csv'))
n_admin_nighttime_id.to_csv(os.path.join(path_to_data,source,'processed',country,'n_admin_nighttime_id.csv'))
print("Done in", round(timer()-start), "sec")

Save
Done in 66 sec


# Figures